In [2]:
%load_ext cython

In [7]:
%%cython

import numpy as np
cimport numpy as cnp

from cpython cimport str

from util cimport datetime64, timedelta64, conversions, floor, ceil

# May change to double depending on what I use
# this function for
cdef inline cnp.int64_t ns_to_days(cnp.int64_t ns):
    # 10^9 ns to 1 sec
    # 60 sec to 1 min
    # 60 min to 1 hour
    # 24 hour to 1 day
    # (10 ** 9) * 60 * 60 * 24
    # 86400000000000 ns = 1 day
    return ns / 86400000000000

cdef inline cnp.int64_t days_to_ns(cnp.int64_t days):
    return days * 86400000000000

cdef split_freq(str freq):
    return int(freq[:-1]), freq[-1]

def resample(frame, freq):
    cdef datetime64[:] index = frame.index.keys_
    cdef datetime64 index0 = index[0]
    
#     cdef datetime64 week_start = index0 - weekday(index0)
#     cdef datetime64 stop = index[-1] + (days_to_ns(7) - weekday(index[-1]))
    
    interval, timeframe = split_freq(freq)

    # dt is in nanoseconds
        
    cdef timedelta64 td = np.timedelta64(1, "W").astype("timedelta64[ns]").astype("int64")
    
    cdef datetime64[:] bins = np.arange(
        start = floor_(index[0], timeframe), 
        stop = ceil_(index[-1], timeframe), 
        step = np.timedelta64(interval, timeframe).astype("timedelta64[ns]").astype("int64")
    )

    cdef splitted = np.split(frame.Open.values, np.cumsum(
        np.bincount(np.digitize(index, bins, right=False))[1:]
    ))[:-1]
    

    cdef int length = len(splitted)
    cdef double[:] ret = np.zeros(length)
    
    for l in range(length):
        ret[l] = np.average(splitted[l])
        
    return ret, bins


Error compiling Cython file:
------------------------------------------------------------
...
import numpy as np
cimport numpy as cnp

from cpython cimport str

from util cimport datetime64, timedelta64, conversions, floor, ceil
^
------------------------------------------------------------

C:\Users\evanw\.ipython\cython\_cython_magic_9327b47deed85fa4491dae26ae9e6e16.pyx:7:0: 'util\floor.pxd' not found

Error compiling Cython file:
------------------------------------------------------------
...
import numpy as np
cimport numpy as cnp

from cpython cimport str

from util cimport datetime64, timedelta64, conversions, floor, ceil
^
------------------------------------------------------------

C:\Users\evanw\.ipython\cython\_cython_magic_9327b47deed85fa4491dae26ae9e6e16.pyx:7:0: 'util\ceil.pxd' not found


In [5]:
%%cython

def add():
    return 5

In [4]:
resample

NameError: name 'resample' is not defined

In [6]:
add()

5